# Подключение библиотек

In [169]:
import numpy as np
import itertools
import random
import sys

# Задание параметров

In [170]:
a = 3;#ширина прохода
b = 1.5;#расстояние между полками
k = 1000;#количество ячеек на стеллаже
n = 20;#количество стеллажей
l = 1;#ширина стеллажа
x = 0.3;#ширина ячейки
N = 5;#количество товаров в заказе

# Список заказов

In [171]:
vertex_ryadi=np.random.randint(1,2*n,N)
vertex_ryadi=np.append(vertex_ryadi[::-1],1)[::-1]
vertex_yacheyki=np.random.randint(1,2*k,N)
vertex_yacheyki=np.append(vertex_yacheyki[::-1],k+1)[::-1]
v=np.column_stack((vertex_ryadi,vertex_yacheyki))
print(v)

[[   1 1001]
 [  21  722]
 [  14   52]
 [  12 1803]
 [  33 1642]
 [   4   17]]


# Создание матрицы

In [172]:
M=np.zeros((N+1,N+1))
for i in range(N+1):
    for j in range(N+1):
        s=0;
        if (i==j):
            M[i][j]=0;
        else:
            # если мы на входе, то отдельно рассчитываем
            if i==1:
                s=s+b;
                if(v[j][1]>k):
                    s=s+a/2;
                else:
                    s=s-a/2;
            if j==1:
                s=s+b;
                if(v[i][1]>k):
                    s=s+a/2;
                else:
                    s=s-a/2;
            # Рассматриваем различные случаи: сверху или снизу, четный или нечётный ряд
            if (v[i][0] % 2 == v[j][0] % 2):
                s=s+abs(v[i][0]-v[j][0])/2*(l+b);
            else:
                s=s+b+abs(v[i][0]-v[j][0])/2*(l+b);    
            if (v[i][1]>k) and (v[j][1]>k):
                s=s+min(v[i][1]-k+v[j][1]-k, 2*k-v[i][1]+2*k-v[j][1])*x
            else:
                if (v[i][1]<k) and (v[j][1]<k):
                    s=s+min(v[i][1]+v[j][1], k-v[i][1]+k-v[j][1])*x
                else:
                    s=s+abs(v[i][1]-v[j][1])*x;
            M[i][j]=s

In [188]:
def distance1(N,i,j,v):
    if j==1:
        if(v[i][1]>k):
            return b+a/2+rasst1(N,i,j,v)
        else:
            return b-a/2+rasst1(N,i,j,v)
    else:
        return rasst1(N,i,j,v)
def rasst1(N,i,j,v):
            if (v[i][0] % 2 == v[j][0] % 2):
                return abs(v[i][0]-v[j][0])/2*(l+b)+rasst2(N,i,j,v);
            else:
                return b+abs(v[i][0]-v[j][0])/2*(l+b)+rasst2(N,i,j,v);  
def rasst2(N,i,j,v):
            if (v[i][1]>k) and (v[j][1]>k):
                return min(v[i][1]-k+v[j][1]-k, 2*k-v[i][1]+2*k-v[j][1])*x
            else:
                if (v[i][1]<k) and (v[j][1]<k):
                    return min(v[i][1]+v[j][1], k-v[i][1]+k-v[j][1])*x
                else:
                    return abs(v[i][1]-v[j][1])*x;
def create_line(N,i,j,v):
    if(j<N):
        if i==j:
            return [0] +create_line(N,i,j+1,v)
        if i==1:
            if(v[j][1]>k):
                return [b+a/2+distance1(N,i,j,v)]+create_line(N,i,j+1,v)
            else:
                return [b-a/2+distance1(N,i,j,v)]+create_line(N,i,j+1,v)
        else:
            return [distance1(N,i,j,v)]+create_line(N,i,j+1,v)
    if (j==N):
        if i==j:
            return [0]
        if i==1:
            if(v[j][1]>k):
                return [b+a/2+distance1(N,i,j,v)]
            else:
                return [b-a/2+distance1(N,i,j,v)]
        else:
            return [distance1(N,i,j,v)]
def create_matrix(N,i,v):
    if(i<N):
        return [create_line(N,i,0,v)]+create_matrix(N,i+1,v)
    else:
        return [create_line(N,i,0,v)]

In [189]:
M1=create_matrix(N,0,v)

In [190]:
print(M1)

[[0, 111.7, 302.45, 256.45, 232.9, 300.45], [111.7, 0, 242.45, 340.05, 294.0, 244.45], [302.45, 242.45, 0, 527.8, 502.25, 33.2], [256.45, 340.05, 527.8, 0, 194.25, 545.8], [232.9, 294.0, 502.25, 194.25, 0, 525.25], [300.45, 244.45, 33.2, 545.8, 525.25, 0]]


In [191]:
print(M)

[[  0.   111.7  302.45 256.45 232.9  300.45]
 [111.7    0.   242.45 340.05 294.   244.45]
 [302.45 242.45   0.   527.8  502.25  33.2 ]
 [256.45 340.05 527.8    0.   194.25 545.8 ]
 [232.9  294.   502.25 194.25   0.   525.25]
 [300.45 244.45  33.2  545.8  525.25   0.  ]]


# Алгоритм Хелда-Карпа

In [192]:
def held_karp(dists):
    n = len(dists)
    # Мэпим каждый набор вершин с путём, который нужно пройти до этого подмножества, а также с предыдущей вершиной. 
    # Наборы вершин представлены битами
    C = {}
    # Задаём пути до каждой вершины из первой вершины
    for k in range(1, n):
        C[(1 << k, k)] = (dists[0][k], 0)
    # Проходим по всем длинам и наборам и запоминаем промежуточные результаты
    for subset_size in range(2, n):
        for subset in itertools.combinations(range(1, n), subset_size):
            # Задаём биты для всех вершин в наборе
            bits = 0
            for bit in subset:
                bits |= 1 << bit
            # Ищем наименьший путь, чтобы добраться до набора
            for k in subset:
                prev = bits & ~(1 << k)
                res = []
                for m in subset:
                    if m == 0 or m == k:
                        continue
                    res.append((C[(prev, m)][0] + dists[m][k], m))
                C[(bits, k)] = min(res)
    bits = (2**n - 1) - 1
    # Ищем наименьший путь
    res = []
    for k in range(1, n):
        res.append((C[(bits, k)][0] + dists[k][0], k))
    opt, parent = min(res)
    # Движемся в обратном направлении, чтобы восстановить весь путь
    path = []
    for i in range(n - 1):
        path.append(parent)
        new_bits = bits & ~(1 << parent)
        _, parent = C[(bits, parent)]
        bits = new_bits
    # Добавляем начальную вершину
    path.append(0)
    path=list(reversed(path))
    path.append(0)
    return opt, path

## Сравнение результатов функционального и классического подходов создания матрицы

In [198]:
dists=M
dist, zakazi = held_karp(dists)

In [199]:
print(dist)

1320.8


In [200]:
dists=M1
dist, zakazi = held_karp(dists)

In [201]:
print(dist)

1320.8


# Подготовка данных к полному перебору

### Функциональный подход

In [203]:
vertexes=list(range(1, N+1))
temp=list(itertools.permutations(vertexes))
def creation(i,t):
    if(i>0):
        return [[0]+list(t[i])+[0]]+creation(i-1,t)
    else: 
        return  [[0]+list(t[0])+[0]]

In [204]:
p=len(temp)-1
routes=creation(p,temp)

# Полный перебор

### Классический подход

In [210]:
min_cost=0
min_route=routes[0]
for j in range(0,len(routes[0])-1):
    min_cost+=M[routes[0][j]][routes[0][j+1]]
for i in routes:
    cost=0;
    for j in range(0,len(i)-1):
        cost+=M[i[j]][i[j+1]]
    if(cost<min_cost):
        min_cost=cost;
        min_route=i;
print(min_cost, min_route)

1320.8 [0, 5, 2, 1, 4, 3, 0]


### Функциональный подход

In [211]:
length=len(routes)
def DIST(M,route):
    if (len(route)>1):
        return M[route[0]][route[1]]+DIST(M,route[1:])
    if(len(route)==1):
        return 0
def MIN_2ROUTE(M,route1,route2):
    if(DIST(M,route1)>=DIST(M,route2)):
        return route2
    else:
        return route1

In [212]:
def MIN_ROUTE(M,Routes):
    if (len(Routes)>1):
        return MIN_2ROUTE(M,Routes[0],MIN_ROUTE(M,Routes[1:]))
    else:
        return Routes[0]         

In [213]:
route=MIN_ROUTE(M,routes)

In [214]:
print(route)

[0, 3, 4, 1, 2, 5, 0]


In [215]:
print(DIST(M,route))

1320.8
